In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
X = pd.read_csv('../glass_formula_a.csv')

df = pd.read_csv('../glass.csv', sep=",")
Y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)
X

RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  cos(RI+Na)  \
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0   -0.854113   
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0   -0.955232   
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0   -0.788898   
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0   -0.556771   
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0   -0.605388   
..       ...    ...   ...   ...    ...   ...   ...   ...  ...         ...   
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0   -0.998662   
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0   -0.745916   
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0   -0.985127   
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0   -0.982278   
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0   -0.999234   

     ...  cos(Si+K)  cos(Si+Ca)  cos(Si+Ba)  cos(Si+Fe)  cos(K+Ca)  cos(K+Ba)  \
0    ...  -0.914457    0.407201   -0.888546   -0.888546  -0.816902   0.998201   
1    ...  -0.578939    0.434414   -0.890037   -0.890037  -0.440377   0.886995   
2    ...  -0.432648    0.612633   -0.742924   -0.742924  -0.310785   0.924909   
3    ...  -0.603136    0.658924   -0.938212   -0.938212  -0.805203   0.841901   
4    ...  -0.196147    0.862094   -0.679754   -0.679754  -0.693271   0.852525   
..   ...        ...         ...         ...         ...        ...        ...   
209  ...  -0.907556    0.994110   -0.156778   -0.938212  -0.986455   0.417595   
210  ...  -0.694286    0.975589    0.732899   -0.694286  -0.519289  -0.019202   
211  ...  -0.396249    0.984095    0.943346   -0.396249  -0.553048  -0.069148   
212  ...  -0.215718    0.917682    0.976284   -0.215718  -0.585923   0.000796   
213  ...  -0.450591    0.955752    0.932968   -0.450591  -0.693271  -0.099041   

     cos(K+Fe)  cos(Ca+Ba)  cos(Ca+Fe)  cos(Ba+Fe)  
0     0.998201   -0.780846   -0.780846    1.000000  
1     0.886995    0.023979    0.023979    1.000000  
2     0.924909    0.073914    0.073914    1.000000  
3     0.841901   -0.357900   -0.357900    1.000000  
4     0.852525   -0.214342   -0.214342    1.000000  
..         ...         ...         ...         ...  
209   0.996802   -0.685707   -0.970191    0.488872  
210   1.000000   -0.844470   -0.519289   -0.019202  
211   1.000000   -0.792913   -0.553048   -0.069148  
212   1.000000   -0.810833   -0.585923    0.000796  
213   1.000000   -0.648471   -0.693271   -0.099041  

[214 rows x 45 columns]

In [10]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# KNN

In [11]:
%%time
plain_sfs = SFS(KNeighborsClassifier(), 
          k_features=(1, 10), 
          forward=True, 
          floating=False,
          scoring='accuracy', 
          cv=10)

plain_sfs.fit(X, Y)

CPU times: user 7.02 s, sys: 37.7 ms, total: 7.06 s
Wall time: 7.07 s


SequentialFeatureSelector(cv=10, estimator=KNeighborsClassifier(),
                          k_features=(1, 10), scoring='accuracy')

In [12]:
plain_sfs.k_score_

0.6874458874458875

In [13]:
selected_features = X.columns[list(plain_sfs.k_feature_idx_)]
x_t = X_train[selected_features]
clf = KNeighborsClassifier()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.42      0.71      0.53         7
           2       0.70      0.64      0.67        11
           3       0.00      0.00      0.00         3
           5       0.60      0.75      0.67         4
           6       0.00      0.00      0.00         2
           7       0.83      0.83      0.83         6

    accuracy                           0.61        33
   macro avg       0.43      0.49      0.45        33
weighted avg       0.55      0.61      0.57        33

0.7073099415204679


# Logistic Regression

In [27]:
plain_sfs_lr = SFS(LogisticRegression(), 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_lr.fit(X, Y)

selected_features = X.columns[list(plain_sfs_lr.k_feature_idx_)]
x_t = X_train[selected_features]
clf = LogisticRegression()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.62      0.77      0.69        13
           2       0.50      0.70      0.58        10
           3       0.00      0.00      0.00         3
           5       1.00      0.50      0.67         2
           6       0.00      0.00      0.00         3
           7       1.00      1.00      1.00         2

    accuracy                           0.61        33
   macro avg       0.52      0.49      0.49        33
weighted avg       0.52      0.61      0.55        33

0.7236842105263157


# Naive Bayes

In [19]:
naive_bayes = GaussianNB()
plain_sfs_nb = SFS(naive_bayes, 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_nb.fit(X, Y) 

SequentialFeatureSelector(cv=10, estimator=GaussianNB(), k_features=(1, 20))

In [20]:
plain_sfs_nb.k_score_

0.6638528138528138

In [21]:
selected_features = X.columns[list(plain_sfs_nb.k_feature_idx_)]
x_t = X_train[selected_features]
clf = GaussianNB()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.33      0.43      0.38         7
           2       0.70      0.64      0.67        11
           3       0.00      0.00      0.00         3
           5       1.00      0.75      0.86         4
           6       1.00      1.00      1.00         2
           7       0.75      1.00      0.86         6

    accuracy                           0.64        33
   macro avg       0.63      0.64      0.63        33
weighted avg       0.62      0.64      0.62        33

0.5517543859649123


# SVM

In [22]:

plain_sfs_svm = SFS(svm.SVC(), 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_svm.fit(X, Y)

SequentialFeatureSelector(cv=10, estimator=SVC(), k_features=(1, 20))

In [23]:
plain_sfs_svm.k_score_

0.7400432900432901

In [24]:
selected_features = X.columns[list(plain_sfs_svm.k_feature_idx_)]
x_t = X_train[selected_features]
clf = svm.SVC()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.40      0.57      0.47         7
           2       0.47      0.64      0.54        11
           3       0.00      0.00      0.00         3
           5       1.00      0.50      0.67         4
           6       1.00      0.50      0.67         2
           7       1.00      0.83      0.91         6

    accuracy                           0.58        33
   macro avg       0.64      0.51      0.54        33
weighted avg       0.60      0.58      0.57        33

0.3593567251461988
